In [29]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import OpenEXR

In [ ]:
RADIUS = 3
SIDE = 1 + 2 * RADIUS

mask = np.zeros((SIDE, SIDE))
mask[RADIUS][RADIUS] = 0

def rotate_ij(direction, i, j):
  match direction:
    case 0: return (i,  j)
    case 1: return (j,  -i)
    case 2: return (-i, -j)
    case 3: return (-j, i)

for direction in range(4):
  for i in range(1, RADIUS+1):
    for j in range(-i, +i):
      ii, jj = rotate_ij(direction, i, j)
      mask[RADIUS + ii][RADIUS + jj] = 5 + direction * 6 + i

px.imshow(mask)

In [49]:
def crop_exr(input_path, output_path, x, y, w, h):
  image = OpenEXR.File(input_path)
  header = image.header()
  w = (np.array([0, 0], dtype=np.int32),
      np.array([w-1, h-1], dtype=np.int32))
  header['dataWindow'] = w
  header['displayWindow'] = w

  def crop_inner(c):
    c = c[y:y+y, x:x+w]
    return np.copy(c) # wouldn't work without copy, OpenEXR bug

  channels = {}
  for name, chan in image.channels().items():
    channels[name] = crop_inner(chan.pixels)

  with OpenEXR.File(header, channels) as outfile:
    outfile.write(output_path)

  #again = OpenEXR.File('cropped.exr')
  #for name, chan in again.channels().items():
  #  display(chan)
  #  px.imshow(chan.pixels).show()

In [ ]:
df = pd.read_csv('../gnorm.csv', header=None)
df[0] = df[0]
display(df.describe())